# Image2Image Pipeline for Stable Diffusion using 🧨 Diffusers 

This notebook shows how to create a custom `diffusers` pipeline for  text-guided image-to-image generation with Stable Diffusion model using  🤗 Hugging Face [🧨 Diffusers library](https://github.com/huggingface/diffusers). 

For a general introduction to the Stable Diffusion model please refer to this [colab](https://colab.research.google.com/github/huggingface/notebooks/blob/main/diffusers/stable_diffusion.ipynb).



# UltraBatchMode

BatchMode (one prompt for many images) * n folders

# Setup

## import GDrive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Check GPU

In [ ]:
!nvidia-smi

## Install libs

In [ ]:
!pip install diffusers==0.3.0 transformers ftfy
!pip install -qq "ipywidgets>=7,<8"

You need to accept the model license before downloading or using the weights. In this post we'll use model version `v1-4`, so you'll need to  visit [its card](https://huggingface.co/CompVis/stable-diffusion-v1-4), read the license and tick the checkbox if you agree. 

You have to be a registered user in 🤗 Hugging Face Hub, and you'll also need to use an access token for the code to work. For more information on access tokens, please refer to [this section of the documentation](https://huggingface.co/docs/hub/security-tokens).

# Token Login

Copy token from GDrive to temp root for auth

In [ ]:
!mkdir -p /root/.huggingface/

In [ ]:
!cp /content/drive/MyDrive/AI/stable_diffusion/token /root/.huggingface/

In [ ]:
from huggingface_hub import notebook_login
import inspect
import warnings
from typing import List, Optional, Union
import torch
from torch import autocast
from tqdm.auto import tqdm
import requests
from io import BytesIO
from PIL import Image
from datetime import datetime
from diffusers import StableDiffusionImg2ImgPipeline

notebook_login()

## Image2Image pipeline.

Load the pipeline

In [ ]:
device = "cuda"
model_path = "CompVis/stable-diffusion-v1-4"

pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
    model_path,
    revision="fp16", 
    torch_dtype=torch.float16,
    use_auth_token=True
)
pipe = pipe.to(device)

Download an initial image and preprocess it so we can pass it to the pipeline.

In [ ]:
# !unzip /content/drive/MyDrive/AI/IMG2IMG_stable_diffusion/init_image/batch_mode/extract_3.zip -d /content/drive/MyDrive/AI/IMG2IMG_stable_diffusion/init_image/batch_mode/

In [ ]:
# !mkdir /content/drive/MyDrive/AI/IMG2IMG_stable_diffusion/$(date +%Y%m%d_%H%M%S) && mv /content/drive/MyDrive/AI/IMG2IMG_stable_diffusion/output/*.png /content/drive/MyDrive/AI/IMG2IMG_stable_diffusion/$(date +%Y%m%d_%H%M%S)

In [ ]:
batch_init_path = "/content/drive/MyDrive/AI/IMG2IMG_stable_diffusion/init_image/batch_mode/"

In [ ]:
from glob import glob
import os

def produce_one_image_by_params(generator, seed, init_image_path, prompt, s_ratio, display_image):
  
  init_img = Image.open(init_image_path).convert("RGB")
  # init_img = init_img.resize((768, 384))


  # init_img = init_img.resize((512, 512))

  init_img = init_img.resize((1024, 512))

  
  filename_w_ext = os.path.basename(init_image_path)
  filename, file_extension = os.path.splitext(filename_w_ext)

  ### black and white option
  # init_img = init_img.convert('L')
  # init_img = init_img.convert('RGB')

  

  now = datetime.now() 
  time = now.strftime("%H%M%S")

  with autocast("cuda"):
      image = pipe(prompt=prompt, init_image=init_img, strength=s_ratio, guidance_scale=7.5, generator=generator).images[0]
  image.save('/content/drive/MyDrive/AI/IMG2IMG_stable_diffusion/output/'+''+'-'+filename+'image.png')
  if display_image:
    display(image)

In [ ]:
# prompt = "byzantine mosaic mosaic high quality impluvium mosaic intricate detailed 4k picture"

In [ ]:
# extension = 'jpg'
# # extension = 'png'

# input_file_paths = glob(batch_init_path+"*/*."+extension, recursive = True)

# # for seed in range(0, 100, 1):
# seed = 7919
# file_dir_list = sorted(input_file_paths)
# # for file_path in tqdm(file_dir_list):
#   # print(file_path)
#   # produce_one_image_by_params(seed, file_path, prompt, 0.4)
#   # produce_one_image_by_params(seed, file_path, prompt, 0.5)
#   # produce_one_image_by_params(seed, file_path, prompt, 0.6)
#   # produce_one_image_by_params(seed, file_path, prompt, 0.75)
#   # produce_one_image_by_params(seed, file_path, prompt, 0.8)
#   # break

In [ ]:
!mkdir /content/drive/MyDrive/AI/IMG2IMG_stable_diffusion/$(date +%Y%m%d_%H%M%S) && mv /content/drive/MyDrive/AI/IMG2IMG_stable_diffusion/output/*.png /content/drive/MyDrive/AI/IMG2IMG_stable_diffusion/$(date +%Y%m%d_%H%M%S)

In [ ]:
# file_dir_list

In [ ]:
# the keys are folder names in the main project folder
# the values are the prompts to be applied to many images


# left_descriptor = 'oil painting mosaic mosaic byzantine mosaic mosaic in the style of greg rutkowski '

left_descriptor = 'a painting of '
right_descriptor = ' fish and other animals on a wall, a mosaic by Pietro Lorenzetti, pixiv, romanesque, academic art, lovecraftian, creative commons attribution'



prompt_per_seq_dict = {


    # --START RUN 1--

    # 'seq1': left_descriptor + 'opening title of the hitchhiker guide to the galaxy' + right_descriptor,
    'seq2': left_descriptor + 'dolphins jumping out of the water' + right_descriptor,
    'seq3': left_descriptor + 'couple of dolphins jumping up and down in the water' + right_descriptor,
    'seq4': left_descriptor + 'dolphin in the air over the water with a garden background and blue sky' + right_descriptor,
    'seq5': left_descriptor + 'dolphin in the air with tips of the trees and blue sky' + right_descriptor,
    'seq6': left_descriptor + 'two dolphins standing on their tail over the water big water splashes' + right_descriptor,
    'seq7': left_descriptor + 'dolphin head out of water smile' + right_descriptor,
    'seq8': left_descriptor + 'group of dolphins swimming in the water from left to right' + right_descriptor,
    
    'seq9': left_descriptor + 'two dolphins and lots of water splashes' + right_descriptor,
    'seq10': left_descriptor + 'two dolphins swimming upright at the water surface' + right_descriptor,
    'seq11': left_descriptor + 'a singular dolphin swimming underwater with water bubbles' + right_descriptor,
    'seq12': left_descriptor + 'a dolphin jumping out of the water with water drops, left to right jump,' + right_descriptor,
    'seq13': left_descriptor + 'men and dolphins standing near water' + right_descriptor,
    'seq14': left_descriptor + 'four dolphins swimming in the same direction' + right_descriptor,

    # --END RUN 1--

    # --START RUN 2--

    'seq15': left_descriptor + 'dolphin sticking head out of water' + right_descriptor,
    'seq16': left_descriptor + 'two dolphins swimming in the same direction' + right_descriptor,
    'seq17': left_descriptor + 'dolphin swimming from left to right' + right_descriptor,
    'seq18': left_descriptor + 'three dolphins mouth open head out of the water' + right_descriptor,
    'seq19': left_descriptor + 'four dolphins swimming in the same direction' + right_descriptor,
    'seq20': left_descriptor + 'dolphin sticking head out of water' + right_descriptor,
    'seq21': left_descriptor + 'underwater multiple dolphins' + right_descriptor,
    'seq22': left_descriptor + 'night full moon flying dolphin water drops' + right_descriptor,
    'seq23': left_descriptor + 'night full moon dolphin backflip into the air water drops' + right_descriptor,
    'seq24': left_descriptor + 'night full moon multiple dolphins flying in the air water drops' + right_descriptor,
    'seq25': left_descriptor + 'full moon reflexion on the water surface in a fine line on the deep blue water' + right_descriptor,
    'seq26': left_descriptor + 'multiple dolphins flying out of the sea with a full moon background' + right_descriptor,
    'seq27': left_descriptor + 'full moon at night with flying dolphins passing in front' + right_descriptor,
    'seq28': left_descriptor + 'full moon in the night sky' + right_descriptor,

    # --END RUN 2--
}


In [ ]:
project_name = "sequenced_dolphin"
batch_init_path + project_name
input_file_paths = glob(batch_init_path+"/*/*", recursive = True)

extension = 'jpg'
seed = 7919

for folder_name in tqdm(prompt_per_seq_dict):

  curr_prompt = prompt_per_seq_dict[folder_name]
  curr_image_list = glob(batch_init_path + project_name + "/"+folder_name+"/*", recursive = True)

  for curr_image in tqdm(sorted(curr_image_list)):
    generator = torch.Generator(device=device).manual_seed(seed)
    pipe.safety_checker = (lambda images, clip_input: (images, False))

    produce_one_image_by_params(generator, seed, curr_image, curr_prompt, 0.6, True)
    break

# simple image resize (bicubic)

In [ ]:
==

In [ ]:
simple_resize_path = "/content/drive/MyDrive/AI/IMG2IMG_stable_diffusion/simple_resize/"

In [ ]:
!mkdir $simple_resize_path

In [ ]:
!cp -r /content/drive/MyDrive/AI/IMG2IMG_stable_diffusion/output/* $simple_resize_path

In [ ]:
from PIL import Image
def simple_upscaling(init_image_path):
  init_img = Image.open(init_image_path).convert("RGB")
  
  ratio = 800/512 #upscaling ratio here

  filename_w_ext = os.path.basename(init_image_path)
  filename, file_extension = os.path.splitext(filename_w_ext)

  init_img = init_img.resize((int(1024*ratio), int(512*ratio)))
  init_img.save(simple_resize_path+''+'-'+filename+'.png')


In [ ]:
input_file_paths = glob(simple_resize_path+"*", recursive = True)
# input_file_paths

In [ ]:
for curr_path in input_file_paths:
  simple_upscaling(curr_path)